# ExoNAMD API v1.0

## Summary

The observer wants to compute the relative and/or absolute NAMD of:
- a given multiplanetary system;
- a subset of multiplanetary systems;
- or all the known ones.

This tool handles all of the above.

In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from datetime import datetime
from datetime import timedelta

pd.options.display.max_columns = 20
pd.options.display.max_rows = 30
pd.options.mode.copy_on_write = True
import warnings

warnings.filterwarnings("ignore")

from exonamd.utils import ROOT
from exonamd.utils import fetch_aliases

### Task 1: getting the data

This task retrieves the parameters of confirmed systems from the NASA Exoplanet Archive database, and stores them in a local database.

In [44]:
# Define the URL for the API
url = "https://exoplanetarchive.ipac.caltech.edu/TAP/sync"

# Define the date you want to filter by
from_scratch = False  # change to False in production
if from_scratch:
    latest = datetime.strptime('1950-01-01', "%Y-%m-%d")  # Example date
else:
    df_old = pd.read_csv(os.path.join(ROOT, "data", "exo.csv"))
    latest = df_old['rowupdate'].max()
    latest = datetime.strptime(latest, "%Y-%m-%d")  # Example date
    latest = latest - timedelta(days=1)

# Convert the date to a string in the format 'YYYY-MM-DD'
latest = latest.strftime('%Y-%m-%d')

# Define the multiplicity you want to filter by
multiplicity = 1

# Define the SQL query to retrieve the required data
query = f"""
SELECT 
    hostname, 
    pl_name, 
    default_flag,
    rowupdate,
    sy_pnum, 
    pl_orbsmax, 
    pl_orbsmaxerr1, 
    pl_orbsmaxerr2, 
    pl_rade,
    pl_radeerr1,
    pl_radeerr2,
    pl_bmasse, 
    pl_bmasseerr1, 
    pl_bmasseerr2, 
    pl_orbeccen, 
    pl_orbeccenerr1, 
    pl_orbeccenerr2, 
    pl_orbincl, 
    pl_orbinclerr1, 
    pl_orbinclerr2,
    pl_trueobliq,
    pl_trueobliqerr1,
    pl_trueobliqerr2
FROM ps
WHERE
    sy_pnum > '{multiplicity}'
    AND rowupdate > '{latest}'
"""

# Define the parameters for the request
params = {
    "query": query,
    "format": "json",
}

# Make the request to the API
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Convert the JSON data to a pandas DataFrame
    df = pd.DataFrame(data)
else:
    print(f"Error: {response.status_code}")

### Task 2: dealing with the aliases

Fetch aliases

In [45]:
aliases = fetch_aliases(df['hostname'].unique())

Fetched 3/3 entries on try 1


Curate aliases

In [46]:
def update_host(row, verbose=False):
    host = row['hostname']
    for key in aliases.keys():
        if host in aliases[key]['host_aliases']:
            if verbose and host != key:
                print(f"Found {host} in aliases, updating to {key}")
            return key
    return host

# # For testing
# df2 = df.copy()
# df2.loc[df2['hostname'] == 'Kepler-20', 'hostname'] = 'KOI-70'
# df2.loc[df2['hostname'] == 'Kepler-80', 'hostname'] = 'KIC 4852528'
# df2['hostname'] = df2.apply(update_host, axis=1)
df['hostname'] = df.apply(update_host, axis=1)

def update_planet(row, verbose=False):
    planet = row['pl_name']
    for key in aliases.keys():
        if planet in aliases[key]['planet_aliases'].keys():
            if verbose and planet != aliases[key]['planet_aliases'][planet]:
                print(f"Found {planet} in aliases, updating to {aliases[key]['planet_aliases'][planet]}")
            return aliases[key]['planet_aliases'][planet]
    return planet

# # For testing
# df2 = df.copy()
# df2.loc[df2['pl_name'] == 'Kepler-20 c', 'pl_name'] = 'KOI-70.01'
# df2.loc[df2['pl_name'] == 'Kepler-11 b', 'pl_name'] = 'KOI-157 b'
# df2['pl_name'] = df2.apply(update_planet, axis=1)
df['pl_name'] = df.apply(update_planet, axis=1)

### Task 3: computing missing values (if any) from simple equations

### Task 4: storing the curated database

In [47]:
df_new = df.copy()

if from_scratch:
    df_latest = df_new
else:
    df_latest = pd.concat([df_new, df_old], ignore_index=True)
    df_latest = df_latest.drop_duplicates(keep='last')

df_latest.to_csv(os.path.join(ROOT, "data", "exo.csv"), index=False)

### Task 5: input missing values (if any) by interpolation

In [ ]:
# code here

### Task 6: storing the curated+interpolated database

In [ ]:
# code here